因为map dataset需要时间比较久（~15min），因此单独进行，然后保存下来，训练的时候只需要直接load 已经map好的dataset就可以了。

In [1]:
import pandas as pd
from datasets import Dataset,load_dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
import torch.nn as nn

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

这里和原代码稍有不同，因为直接使用load_dataset方法load txt文件在我的环境里会有问题，不报错也不运行。因此改成了通过pandas中转一下。

根据原代码里给出的wiki句子数据集的下载路径下好数据集（ https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/wiki1m_for_simcse.txt ），然后 wiki_text_file = 你的数据集路径。

In [3]:
# use pandas to read simCSE-wiki.txt
wiki_text_file = 'your-path-to/simCSE-wiki.txt'
wiki = pd.read_csv(wiki_text_file,sep = '\t',header = None)
wiki.columns = ['text']
# use Dataset.from_pandas to convert to dataset
wiki_dataset = Dataset.from_pandas(wiki,split= "train")
wiki_dataset

Dataset({
    features: ['text'],
    num_rows: 995447
})

In [6]:
def prepare_features(examples):
    
    total = len(examples['text'])
    # total = batch_size
    
    # Avoid "None" fields 
    for idx in range(total):
        if examples['text'][idx] is None:
            examples['text'][idx] = " "
        if examples['text'][idx] is None:
            examples['text'][idx] = " "

    sentences = examples['text'] + examples['text']

    # set max_length here:
    sent_features = tokenizer(sentences, max_length=32, truncation=True, padding="max_length")

    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]
    
    return features

In [7]:
train_dataset = wiki_dataset.map(prepare_features,batched=True, remove_columns=['text'], batch_size=4000)

  0%|          | 0/249 [00:00<?, ?ba/s]

In [8]:
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])

In [9]:
train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'token_type_ids'],
    num_rows: 995447
})

In [10]:
# save to disk for reuse
train_dataset.save_to_disk("wiki_for_sts_32")